# 05 GPU 효율적인 학습
## 5.1 GPU에 올라가는 데이터 살펴보기
<br>
자주 발생하는 OOM 오류. GPU 메모리에 데이터가 가득 차 더 이상 새로운 데이터를 추가하지 못해 발생. <br>

### 5.1.1 딥러닝 모델의 데이터 타입
### 5.1.2 양자화로 모델 용량 줄이기
양자화: 적은 비트로 모델을 표현. 양자화 기술에서는 더 적은 비트를 사용하면서도 원본 데이터의 정보를 최대한 소실 없이 유지하는 것이 핵심 <br>
### 5.1.3 GPU 메모리 분해하기
__딥러닝 학습과정__
- 순전파 수행
- 순전파에서 계산한 손실로부터 역전파 수행
- 옵티마이저로 모델 업데이트 <br>
*역전파를 수행하기 위해 저장하고 잇는 값들이 순전파 상태 값 <br>
*그레디언트는 역전파 결과 생성

__GPU 메모리에 저장되는 데이터__
- 모델 파라미터
- 그레디언트(gradient)
- 옵티마이저 상태(optimizer state)
- 순전파 상태(forward activation)
<br>

fp16모델 기준, AdamW와 같은 옵티마이저 사용 시 학습에 필요한 최소한의 메모리
- 모델 파라미터: 2바이트 * 파라미터 수(B, 10억 개) = N
- 그레디언트: 2바이트 * 파라미터 수(B, 10억 개) = N
- 옵티마이저 상태: 2바이트 * 파라미터 수(B, 10억 개) * 2(상태 수) = 2N <br>
*모두 합쳐 대략 4N의 메모리가 필요 <br>
*순전파 상태(배치 크기, 시퀀스 길이, 잠재 상태 크기에 따라 달라짐)를 저장하기 위한 메모리 추가로 필요

In [1]:
# 메모리 사용량 측정을 위한 함수
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요")

print_gpu_utilization()

런타임 유형을 GPU로 변경하세요


In [2]:
# 모델을 불러오고 GPU 메모리와 데이터 타입 확인
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"": 0})
    
    print_gpu_utilization()
    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id)
print("모델 파라미터 데이터 타입: ", model.dtype)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Cannot access accelerator device when none is available.